In [1]:
#%pip install -U torch
#%pip install -U pandas
#%pip install -U matplotlib
#%pip install -U seaborn
#%pip install -U trl
#%pip install -U numpy
#%pip install -U peft
%pip install -U transformers
%pip install -U datasets
%pip install -U bitsandbytes #==0.41.1
%pip install -U huggingface_hub
#%pip install -U tqdm
%pip install -U codecarbon
%pip install -U ipywidgets
%pip install -U accelerate

Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 12.3 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: datasets
    Found existing installation: datasets 2.19.1
    Uninstalling datasets-2.19.1:
      Successfully uninstalled datasets-2.19.1
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import accelerate
from accelerate import infer_auto_device_map
#from datasets import load_dataset, Dataset
#from scipy.special import softmax
#from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model
#from sklearn.metrics import accuracy_score, f1_score, log_loss, confusion_matrix
from transformers import  AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
from huggingface_hub import notebook_login, login
#import json
import pandas as pd
import os
#from tqdm import tqdm
#import random
from codecarbon import EmissionsTracker, track_emissions
from time import time
import csv
from get_tokens import *

In [3]:
# Check if GPUs are available and move model to GPUs
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [4]:
model_name = "meta-llama/Meta-Llama-3-8B-Instruct"
#model_name = "meta-llama/Meta-Llama-3-70B-Instruct"
#model_name = "/kaggle/input/llama-3/transformers/8b-chat-hf/1"

#model_name = 'meta-llama/Meta-Llama-3-8B'
#model_name = 'meta-llama/Llama-2-7b-chat-hf'
#model_name = 'meta-llama/Llama-2-13b-chat-hf'
#model_name = 'meta-llama/Llama-2-70b-chat-hf'
#model_name = "/kaggle/input/llama-3/transformers/8b-hf/1"

#model_name = "unsloth/llama-3-70b-bnb-4bit"

#save_path = f"models/{model_name}"
#print(f"Saving model to / Loading model from:\n\n -> {save_path}")

In [5]:
#quantization configurations - so you quantize the model while inferencing
bnb_config = BitsAndBytesConfig(
    #load_in_4bit = True,
    load_in_8bit=True,
    llm_int8_skip_modules=["lm_head"],
    #llm_int8_threshold=4.0,
    #llm_int8_has_fp16_weight=True,
    #bnb_4bit_qunat_type = "nf4",
    #bnb_4bit_compute_dtype = torch.float16,
)

In [6]:
login(token=get_hf_token())
#notebook_login()

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /home/ubuntu/.cache/huggingface/token
Login successful


In [7]:
#device_map = infer_auto_device_map(model, max_memory={0: "20GiB", 1: "20GiB", 2: "20GiB", 3: "20GiB", "cpu": "100GiB"})

In [8]:
# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.bos_token

# Load the model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    #load_in_8bit=True,
    #model_path,
    #num_labels=4,  # Adjust if necessary based on the specific task
    quantization_config=bnb_config,
    device_map="auto",
    #device_map="sequential",
    #torch_dtype=torch.float16,
    #device_map=device_map,
    #load_in_8bit_fp32_cpu_offload=True,
)
model.config.pad_token_id = tokenizer.pad_token_id

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

# Test the Model

In [9]:
# List of words that should be used to create sentences.

words = [
    "apple", "book", "car", "dog", "elephant", "forest", "guitar", "house", 
    "island", "jacket", "kangaroo", "lamp", "mountain", "notebook", "ocean", 
    "pencil", "queen", "river", "star", "tree", "umbrella", "village", 
    "window", "xylophone", "yacht", "zebra", "balloon", "camera", "desert", 
    "engine", "flower", "garden", "honey", "iceberg", "jungle", "kite", 
    "ladder", "moon", "nest", "octopus", "pirate", "quilt", "robot", "swan", 
    "telescope", "unicorn", "violin", "whale", "x-ray", "laptop"
]

In [10]:
# List of examples to be provided in the system prompt

example_sentences = [
    "Why did the bicycle fall over? Because it was two-tired!",
    "Why don't scientists trust atoms? Because they make up everything!",
    "Why did the scarecrow win an award? Because he was outstanding in his field!",
    "Why don't skeletons fight each other? They don't have the guts!",
    "Why did the computer go to the doctor? Because it had a virus!",
    "Why was the math book sad? Because it had too many problems!",
    "Why did the coffee file a police report? It got mugged!",
    "Why did the tomato turn red? Because it saw the salad dressing!",
    "Why don't eggs tell jokes? They might crack up!",
    "Why did the golfer bring two pairs of pants? In case he got a hole in one!",
    "Why do cows wear bells? Because their horns don't work!",
    "Why don't some couples go to the gym? Because some relationships don't work out!",
    "Why did the photo go to jail? It was framed!",
    "Why don't programmers like nature? It has too many bugs!",
    "Why did the bicycle stand up by itself? It was two-tired!",
    "Why did the music teacher need a ladder? To reach the high notes!",
    "Why did the cookie go to the doctor? Because it felt crummy!",
    "Why did the student eat his homework? Because his teacher told him it was a piece of cake!",
    "Why don't oysters donate to charity? Because they are shellfish!",
    "Why did the broom get a promotion? Because it swept the competition!",
    "Why don't we see elephants hiding in trees? Because they are so good at it!",
    "Why did the fish blush? Because it saw the ocean's bottom!",
    "Why did the barber win the race? Because he knew all the shortcuts!",
    "Why did the banana go to the doctor? Because it wasn't peeling well!",
    "Why don't dinosaurs talk? Because they are extinct!",
    "Why did the clock go back to school? To learn about time management!",
    "Why did the farmer win an award? Because he was outstanding in his field!",
    "Why did the astronaut break up with his girlfriend? He needed space!",
    "Why did the shoe go to the party alone? Because it didn't want to be a pair!",
    "Why did the tree go to the dentist? To get its roots checked!",
    "Why did the calendar go on a diet? It wanted to lose some days!",
    "Why did the stadium get hot? All the fans left!",
    "Why did the belt go to jail? It held up a pair of pants!",
    "Why did the cookie cry? Because its mom was a wafer too long!",
    "Why did the cow jump over the moon? To get to the milky way!",
    "Why did the skeleton go to the party alone? He had no body to go with!",
    "Why did the grape stop in the middle of the road? Because it ran out of juice!",
    "Why did the bee get married? Because he found his honey!",
    "Why did the soccer ball quit the team? It was tired of being kicked around!",
    "Why did the traffic light turn red? You would too if you had to change in the middle of the street!",
    "Why did the scarecrow become a successful neurosurgeon? He was outstanding in his field!",
    "Why did the chicken cross the playground? To get to the other slide!",
    "Why did the belt get a promotion? It was a cinch!",
    "Why did the teacher wear sunglasses? Because her students were so bright!",
    "Why did the clock get kicked out of class? It was tocking too much!",
    "Why did the frog call his insurance company? He had a jump in his car!",
    "Why did the keyboard get a speeding ticket? It had a problem with the space bar!",
    "Why did the painting go to art school? It wanted to brush up on its skills!",
    "Why did the scientist install a knocker on his door? He wanted to win the No-bell prize!",
    "Why did the tomato turn green? Because it was embarrassed to ketchup!",
    "Why did the pencil go to jail? It was caught in a sketchy situation!",
    "Why did the music note need a loan? It needed some major funding!",
    "Why did the fisherman put peanut butter into the sea? To go with the jellyfish!",
    "Why did the bicycle bring a map? Because it didn't want to get lost on its wheel-y big adventure!",
    "Why did the pizza go to the party? It wanted to slice up the dance floor!",
    "Why did the phone sit on a bench? It wanted to recharge its batteries!",
    "Why did the blanket get arrested? It was covering up a crime!",
    "Why did the chef go to jail? Because he beat the eggs and whipped the cream!",
    "Why did the sandwich go to the beach? To get a little bologna-sun!",
    "Why did the banker switch careers? He lost interest!",
    "Why did the shovel go to therapy? It had too much dirt on its mind!",
    "Why did the soccer player bring string to the game? So he could tie the score!",
    "Why did the alarm clock break up with the pillow? It couldn't handle the pressure!",
    "Why did the frog take the bus to work? His car got toad away!",
    "Why did the dentist become a gardener? He wanted to brush up on his roots!",
    "Why did the light bulb fail his test? He wasn't too bright!",
    "Why did the ocean break up with the shore? It needed some space to tide things over!",
    "Why did the shoe store close down? It lost its sole!",
    "Why did the banana go out with the prune? Because it couldn't find a date!",
    "Why did the carpenter become a musician? He wanted to nail every note!"
]


# Approach with Pipeline

In [11]:
# Initialize the pipeline
text_generation_pipeline = pipeline('text-generation', model=model, tokenizer=tokenizer, device_map="auto")

In [12]:
def query_model(word, temperature=0.6, max_length=48, num_examples=1, multiply_by=1):
    start_time = time()
    

    system_message = f"""
    You are an AI assistant designed to tell jokes for a given word.
    Generate {num_examples*multiply_by} short joke(s) that are exactly 2 short sentences long. Do not include any follow-up questions or explanations.

    For example, a joke for the word "yogurt":
    'Why did the yogurt go to the art gallery? Because it wanted to be cultured!'

    Here are a few further examples: 
    - 'Why did the bicycle fall over? Because it was two-tired!'
    - 'Why don't scientists trust atoms? Because they make up everything!'

    
    Be as close as possible to the example jokes!
    Respond with only the {num_examples*multiply_by} joke(s) itself. Do not include any introductory phrases.

    """
    
    user_message = "Question: " + f'Now, tell me {num_examples*multiply_by} short joke(s) for the word: "{word}"\n' + " Answer:"
    
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_message},
    ]
    prompt = tokenizer.apply_chat_template(
        messages, 
        tokenize=False, 
        add_generation_prompt=True
    )
    
    # Tokenize the prompt to count input tokens
    input_tokens = tokenizer.encode(prompt)
    input_token_count = len(input_tokens)
    
    terminators = [
        tokenizer.eos_token_id,
        tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]
    sequences = text_generation_pipeline(
        prompt,
        do_sample=True,
        top_p=0.9,
        temperature=temperature,
        eos_token_id=terminators,
        max_new_tokens=max_length,
        return_full_text=False,
        pad_token_id=model.config.eos_token_id
    )
    answer = sequences[0]['generated_text']
    
    # Tokenize the answer to count output tokens
    output_tokens = tokenizer.encode(answer)
    output_token_count = len(output_tokens)
    
    end_time = time()
    ttime = f"Total time: {round(end_time-start_time, 2)} sec."

    return (
        user_message + "\n\n" + answer + "\n\n" + ttime,
        input_token_count,
        output_token_count
    )

In [13]:
avg_time = 10
avg_in_tok = 103
avg_out_tok = 30

model_name = "outputtest-meta-llama/Meta-Llama-3-8B-Instruct-OutputTest"

#avg_time = 4.5
#avg_in_tok = 103
#avg_out_tok = 25

# Number of runs
n = 3 # Change to 5 for all the other tests
num_words = len(words)
total_prompts = n*num_words

example_counts = [1, 5, 10, 20, 30]
#multiply = [1, 3, 5] # 5 did not work yet, because GPU 1 had too much in memory. Maybe try with other GPU map

#num_examples = 70
multiply_by = 1

for num_examples in example_counts:

    total_input_tokens = 0
    total_output_tokens = 0
    max_length = num_examples * multiply_by * 30

    print("="*10 + f" INFERENCE TEST with {num_examples * multiply_by} EXAMPLES " + "="*10 + 
        "\n\n" + 
        f"""
        Starting Test with {n} Runs and {num_words} Words/Run. \n
        Total Prompts: {total_prompts}\n\n
        Expected Runtime: {(total_prompts*avg_time)/60}min \n
        Expected Input Tokens: {avg_in_tok*total_prompts} \n
        Expected Output Tokens: {avg_out_tok*total_prompts} \n\n
        """)

    tracker = EmissionsTracker(project_name=f'{model_name}-emissiondata-{num_examples * multiply_by}-examples', log_level="error", pue = 1.22)
    tracker.start()

    # Run the function for each word in the array
    for i in range(n):
        print("="*25 + f" Run {i+1} " + "="*25)
        print("\n")
        for idx, word in enumerate(words):
            print("-"*15 + f" Run {i+1} - Word {idx+1} " + "-"*15)
            output, input_token_count, output_token_count = query_model(word, max_length=max_length, num_examples=num_examples, temperature=0.2, multiply_by=multiply_by)
            
            total_input_tokens += input_token_count
            total_output_tokens += output_token_count
            
            print(output)
            print("\n")
            print(f"Input tokens: {input_token_count}, Output tokens: {output_token_count}")
            print("\n")
        print("\n")
    
    emissions: float = tracker.stop()

    em_i = emissions/total_input_tokens
    em_o = emissions/total_output_tokens
    em_p = emissions/total_prompts


    print("="*15 + f" RESULTS for {num_examples * multiply_by} EXAMPLES " + "="*15 + 
        "\n\n" + 
        f"""
        Finished {n} Runs with {num_words} Words/Run.\n\n
        Total Prompts: {total_prompts}\n
        Total Input Tokens: {total_input_tokens}, AVG/Prompt: {total_input_tokens/total_prompts}\n
        Total Output Tokens: {total_output_tokens}, AVG/Prompt: {total_output_tokens/total_prompts}\n
        """ + 
        
        "-"*50 + "\n" +
        
        f"""
        Total Inference Emissions: {emissions:.3f}kg CO₂eq\n\n
        Emissions / 1.000.000 Input Tokens: {em_i * 1000000:.3f}kg CO₂eq\n
        Emissions / 1.000.000 Output Tokens: {em_o * 1000000:.3f}kg CO₂eq\n
        Emissions / 1.000 Prompts: {em_p * 1000:.3f}kg CO₂eq\n

        """
        )

    # Save results to a CSV file
    results = [
        ["Runs", n],
        ["Words per Run", num_words],
        ["Total Prompts", total_prompts],
        ["Total Input Tokens", total_input_tokens],
        ["Avg Input Tokens per Prompt", round(total_input_tokens / total_prompts, 3)],
        ["Total Output Tokens", total_output_tokens],
        ["Avg Output Tokens per Prompt", round(total_output_tokens / total_prompts, 3)],
        ["Total Emissions (kg CO₂eq)", round(emissions, 3)],
        ["Emissions per Million Input Tokens (kg CO₂eq)", round(em_i * 1000000, 3)],
        ["Emissions per Million Output Tokens (kg CO₂eq)", round(em_o * 1000000, 3)],
        ["Emissions per Thousand Prompts (kg CO₂eq)", round(em_p * 1000, 3)]
    ]

    # Ensure the directory exists
    output_file_path = f'{model_name}-emissiondata-{num_examples * multiply_by}-examples.csv'
    os.makedirs(os.path.dirname(output_file_path), exist_ok=True)

    with open(output_file_path, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(["Metric", "Value"])
        writer.writerows(results)

    print(f"Results saved to {output_file_path}")

========== INFERENCE TEST with 1 EXAMPLES ==========


        Starting Test with 3 Runs and 50 Words/Run. 

        Total Prompts: 150


        Expected Runtime: 25.0min 

        Expected Input Tokens: 15450 

        Expected Output Tokens: 4500 


        


========================= Run 1 =========================


--------------- Run 1 - Word 1 ---------------
Question: Now, tell me 1 short joke(s) for the word: "apple"
 Answer:

Why did the apple join the gym? To get some core strength!

Total time: 3.33 sec.


Input tokens: 186, Output tokens: 15


--------------- Run 1 - Word 2 ---------------
Question: Now, tell me 1 short joke(s) for the word: "book"
 Answer:

Why did the book go to therapy? Because it had a bad chapter!

Total time: 2.11 sec.


Input tokens: 186, Output tokens: 16


--------------- Run 1 - Word 3 ---------------
Question: Now, tell me 1 short joke(s) for the word: "car"
 Answer:

Why did the car go to the doctor? Because it was feeling a little car-sick!

Total time: 2.6 sec.


Input tokens: 186, Output tokens: 20


--------------- Run 1 - Word 4 ---------------
Question: Now, tell me 1 short joke(s) for the word: "dog"
 Answer:

Why did the dog go to the vet? Because it was feeling ruff!

Total time: 2.27 sec.




You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Question: Now, tell me 1 short joke(s) for the word: "jacket"
 Answer:

Why did the jacket go to therapy? Because it was feeling a little worn out!

Total time: 2.4 sec.


Input tokens: 187, Output tokens: 18


--------------- Run 1 - Word 11 ---------------
Question: Now, tell me 1 short joke(s) for the word: "kangaroo"
 Answer:

Why did the kangaroo quit his job? Because he was hopping mad about the working conditions!

Total time: 2.65 sec.


Input tokens: 188, Output tokens: 20


--------------- Run 1 - Word 12 ---------------
Question: Now, tell me 1 short joke(s) for the word: "lamp"
 Answer:

Why did the lamp go to therapy? Because it was feeling a little dim!

Total time: 2.26 sec.


Input tokens: 186, Output tokens: 17


--------------- Run 1 - Word 13 ---------------
Question: Now, tell me 1 short joke(s) for the word: "mountain"
 Answer:

Why did the mountain go to therapy? Because it was feeling a little peaked!

Total time: 2.24 sec.


Input tokens: 187, Output tokens: 17


KeyboardInterrupt: 

In [ ]:
avg_time = 10
avg_in_tok = 103
avg_out_tok = 30

#avg_time = 4.5
#avg_in_tok = 103
#avg_out_tok = 25

# Number of runs
n = 3 # Change to 5 for all the other tests
num_words = len(words)
total_prompts = n*num_words

example_counts = [1, 5, 10, 20, 30]
multiply = [1, 3, 5] # 5 did not work yet, because GPU 1 had too much in memory. Maybe try with other GPU map

num_examples = 40


for multiply_by in multiply:

    total_input_tokens = 0
    total_output_tokens = 0
    max_length = num_examples * multiply_by * 30

    print("="*10 + f" INFERENCE TEST with {num_examples * multiply_by} EXAMPLES " + "="*10 + 
        "\n\n" + 
        f"""
        Starting Test with {n} Runs and {num_words} Words/Run. \n
        Total Prompts: {total_prompts}\n\n
        Expected Runtime: {(total_prompts*avg_time)/60}min \n
        Expected Input Tokens: {avg_in_tok*total_prompts} \n
        Expected Output Tokens: {avg_out_tok*total_prompts} \n\n
        """)

    tracker = EmissionsTracker(project_name=f'{model_name}-emissiondata-{num_examples * multiply_by}-examples', log_level="error", pue = 1.22)
    tracker.start()

    # Run the function for each word in the array
    for i in range(n):
        print("="*25 + f" Run {i+1} " + "="*25)
        print("\n")
        for idx, word in enumerate(words):
            print("-"*15 + f" Run {i+1} - Word {idx+1} " + "-"*15)
            output, input_token_count, output_token_count = query_model(word, max_length=max_length, num_examples=num_examples, temperature=0.2, multiply_by=multiply_by)
            
            total_input_tokens += input_token_count
            total_output_tokens += output_token_count
            
            print(output)
            print("\n")
            print(f"Input tokens: {input_token_count}, Output tokens: {output_token_count}")
            print("\n")
        print("\n")
    
    emissions: float = tracker.stop()

    em_i = emissions/total_input_tokens
    em_o = emissions/total_output_tokens
    em_p = emissions/total_prompts


    print("="*15 + f" RESULTS for {num_examples * multiply_by} EXAMPLES " + "="*15 + 
        "\n\n" + 
        f"""
        Finished {n} Runs with {num_words} Words/Run.\n\n
        Total Prompts: {total_prompts}\n
        Total Input Tokens: {total_input_tokens}, AVG/Prompt: {total_input_tokens/total_prompts}\n
        Total Output Tokens: {total_output_tokens}, AVG/Prompt: {total_output_tokens/total_prompts}\n
        """ + 
        
        "-"*50 + "\n" +
        
        f"""
        Total Inference Emissions: {emissions:.3f}kg CO₂eq\n\n
        Emissions / 1.000.000 Input Tokens: {em_i * 1000000:.3f}kg CO₂eq\n
        Emissions / 1.000.000 Output Tokens: {em_o * 1000000:.3f}kg CO₂eq\n
        Emissions / 1.000 Prompts: {em_p * 1000:.3f}kg CO₂eq\n

        """
        )

    # Save results to a CSV file
    results = [
        ["Runs", n],
        ["Words per Run", num_words],
        ["Total Prompts", total_prompts],
        ["Total Input Tokens", total_input_tokens],
        ["Avg Input Tokens per Prompt", round(total_input_tokens / total_prompts, 3)],
        ["Total Output Tokens", total_output_tokens],
        ["Avg Output Tokens per Prompt", round(total_output_tokens / total_prompts, 3)],
        ["Total Emissions (kg CO₂eq)", round(emissions, 3)],
        ["Emissions per Million Input Tokens (kg CO₂eq)", round(em_i * 1000000, 3)],
        ["Emissions per Million Output Tokens (kg CO₂eq)", round(em_o * 1000000, 3)],
        ["Emissions per Thousand Prompts (kg CO₂eq)", round(em_p * 1000, 3)]
    ]

    # Ensure the directory exists
    output_file_path = f'{model_name}-emissiondata-{num_examples * multiply_by}-examples.csv'
    os.makedirs(os.path.dirname(output_file_path), exist_ok=True)

    with open(output_file_path, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(["Metric", "Value"])
        writer.writerows(results)

    print(f"Results saved to {output_file_path}")

# Approach without Pipeline

In [ ]:
# Write a Text Generation Function
#@track_emissions(project_name=model_name)
#def generate_text(prompt, max_length=50):
#    inputs = tokenizer(prompt, return_tensors="pt").to(device)
#    with torch.no_grad():
#        outputs = model.generate(
#            **inputs, 
#            #max_length=max_length,
#            pad_token_id=tokenizer.eos_token_id,
#            max_new_tokens=max_length
#        )
#    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [ ]:
#def create_prompt(word): 
#    prompt = f"""
#    You are an AI-Assistant that tells jokes for a given word.
#    Generate a short joke that is exactly 2 sentences long.
#    Do not include any follow-up questions or explanations.#
#
#    For example, a joke for the word "yogurt":
#    'Why did the yogurt go to the art gallery? Because it wanted to be cultured!'#
#
#    Now, tell me one short joke for the word "{word}".
#    """
#    return prompt

In [ ]:
#tracker = EmissionsTracker(project_name=model_name, log_level="error", pue = 1.22)#, output_file=f'{model_name}_emissions.csv')
#tracker.start()

# Number of runs
#n = 1

# Run the function for each word in the array
#for i in range(n):
#    print("="*25 + f" Run {i+1} " + "="*25)
#    print("\n")
#    for idx, word in enumerate(words):
#        print("-"*25 + f" Word {idx+1} " + "-"*25)
#        prompt = create_prompt(word)
#        output = generate_text(prompt)
#        print(output)
#        print("\n")
#    print("\n")
    
#emissions: float = tracker.stop()
#print(emissions)